In [1]:
import torch 
import numpy as np
import pandas as pd
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import joblib

from sklearn.model_selection import train_test_split



In [2]:
dataset2 = pd.read_csv(r'C:\Users\Kulde\OneDrive\Desktop\OFFICE\CAR_PREDICT\data\processed\cleaned_car_csv.csv')
dataset2

,price,year,manufacturer,model,fuel,odometer,title_status,transmission,type,paint_color,state
0,33590,2014,gmc,sierra 1500 crew,gas,57923,clean,other,pickup,white,al
1,22590,2010,chevrolet,silverado 1500,gas,71229,clean,other,pickup,blue,al
2,39590,2020,chevrolet,silverado 1500 crew,gas,19160,clean,other,pickup,red,al
3,30990,2017,toyota,tundra double cab,gas,41124,clean,other,pickup,red,al
4,15000,2013,ford,f-150 xlt,gas,128000,clean,automatic,truck,black,al
...,...,...,...,...,...,...,...,...,...,...,...
169006,30990,2018,mercedes-benz,glc 300 sport,gas,15080,clean,automatic,other,white,wy
169007,33590,2018,lexus,gs 350 sedan,gas,30814,clean,automatic,sedan,white,wy
169008,30590,2020,volvo,s60 t5 momentum,gas,12029,clean,other,sedan,red,wy
169009,34990,2020,cadillac,xt4 sport suv,diesel,4174,clean,other,hatchback,white,wy


In [11]:
dataset2['odometer'].max()

299999

In [15]:


model_counts = dataset2['paint_color'].value_counts().head(100)
# Extract the model names as a list
model_names = model_counts.index.tolist()
print(model_names)

['white', 'black', 'silver', 'blue', 'red', 'grey', 'green', 'custom', 'brown', 'yellow', 'orange', 'purple']


In [ ]:
dataset2

In [3]:

x_train_full,x_test,y_train_full,y_test = train_test_split(dataset2.drop(columns=['price']),dataset2['price'],test_size=0.2,random_state=42)

scaler_y = StandardScaler()
y_train_full = scaler_y.fit_transform(pd.DataFrame(y_train_full))
y_test = scaler_y.transform(pd.DataFrame(y_test))

target_encoder = TargetEncoder()
x_train_full = target_encoder.fit_transform(x_train_full,y_train_full)
x_test = target_encoder.transform(x_test)

scaler_x = StandardScaler()
x_train_full = scaler_x.fit_transform(x_train_full)
x_test = scaler_x.transform(x_test)

In [4]:
x_train,x_val,y_train,y_val = train_test_split(x_train_full,y_train_full,test_size=0.2,random_state=42)

X_train_tensor = torch.from_numpy(x_train.astype(np.float32))
X_val_tensor = torch.from_numpy(x_val.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_val_tensor = torch.from_numpy(y_val.astype(np.float32))
x_test_tensor = torch.from_numpy(x_test.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

In [5]:
class CustomDataset(Dataset):
  def __init__(self, features, labels):
    self.features = features
    self.labels = labels
  def __len__(self):
    return len(self.features)
  def __getitem__(self, idx):
    return self.features[idx], self.labels[idx]

train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
val_dataset = CustomDataset(X_val_tensor, y_val_tensor)
test_dataset = CustomDataset(x_test_tensor,y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
class Model1(nn.Module):
    def __init__(self,num_features):
        super().__init__()
        self.linear = nn.Linear(num_features,128)
        self.relu = nn.ReLU()
        self.linear1 = nn.Linear(128,1)
    def forward(self,features):
        out = self.linear(features)
        out = self.relu(out)
        out = self.linear1(out)
        return out
    
model = Model1(X_train_tensor.shape[1])
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_function = nn.MSELoss()

best_val_loss = float('inf')  # Start with a very large value for validation loss
best_model_wts = None
patience = 2  # Reduce patience to stop earlier if no improvement
patience_counter = 0  # Counter to track epochs without improvement

epochs = 20
patience = 3  # Example patience for early stopping
best_val_loss = float('inf')
patience_counter = 0

for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    val_loss = 0.0
    val_true = []
    val_pred = []

    for train_batch, val_batch in zip(train_loader, val_loader):
        # Training phase
        train_features, train_labels = train_batch
        y_pred = model(train_features)
        train_loss = loss_function(y_pred, train_labels.view(-1, 1))

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        total_loss += train_loss.item()

        # Validation phase
        model.eval()
        with torch.no_grad():
            val_features, val_labels = val_batch
            val_outputs = model(val_features)
            batch_val_loss = loss_function(val_outputs, val_labels.view(-1, 1))

            val_loss += batch_val_loss.item()
            val_true.extend(val_labels.numpy())
            val_pred.extend(val_outputs.numpy())

    avg_loss = total_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)

    val_true = np.array(val_true)
    val_pred = np.array(val_pred)
    val_r2 = r2_score(val_true, val_pred)


    # Early stopping logic
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model_wts = model.state_dict()  # Save the best model
        patience_counter = 0  # Reset patience counter
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print(f"Early stopping after epoch {epoch + 1}.")
        break


In [8]:

# Load best model weights if early stopping triggered

# Evaluate the model on the test set
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        outputs = model(batch_features)
        y_true.extend(batch_labels.numpy())
        y_pred.extend(outputs.numpy())

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate test set performance metrics
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print(f"\nTest Set Performance Metrics:")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R² Score: {r2:.4f}")



Test Set Performance Metrics:
Mean Squared Error (MSE): 0.2400
Root Mean Squared Error (RMSE): 0.4899
Mean Absolute Error (MAE): 0.3406
R² Score: 0.7592


In [5]:
from torchsummary import summary

# Assuming model is already defined
summary(model, input_size=(X_train_tensor.shape[1],))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 128]           1,408
              ReLU-2                  [-1, 128]               0
            Linear-3                    [-1, 1]             129
Total params: 1,537
Trainable params: 1,537
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.01
Estimated Total Size (MB): 0.01
----------------------------------------------------------------


In [6]:
total_params = sum(p.numel() for p in model.parameters())
param_size_bytes = total_params * 4  # Assuming 32-bit (4 bytes) floating point numbers
param_size_mb = param_size_bytes / (1024**2)  # Convert bytes to MB
print(f"Model size (parameters): {param_size_mb:.2f} MB")


Model size (parameters): 0.01 MB
